In [1]:
import pandas as pd
from readresults.readres_mvpa import merge_results, parse_roi_info, exclude_participants
%load_ext autoreload
%autoreload 2

In [2]:
res_list = ['../../../MVPA_results/try_newformat.csv']
results = merge_results(res_list)

In [3]:
results = parse_roi_info(results)
results = exclude_participants(results)

In [4]:
results

,correct,output,rawdistance,distance,runno,subject,roi,approach,traindataformat,testdataformat,traintask,testtask,trainmodel,testmodel,hemi,contrast,nvoxels
0,1,0,2.198136,0.439182,1,sub-001,ba-17-18,traintest,betas,betas,train,test,6,3,L,none,none
1,0,0,1.209115,-0.240386,1,sub-001,ba-17-18,traintest,betas,betas,train,test,6,3,L,none,none
2,1,0,3.995326,0.800423,2,sub-001,ba-17-18,traintest,betas,betas,train,test,6,3,L,none,none
3,0,0,3.825320,-0.766251,2,sub-001,ba-17-18,traintest,betas,betas,train,test,6,3,L,none,none
4,1,0,3.472912,0.695416,3,sub-001,ba-17-18,traintest,betas,betas,train,test,6,3,L,none,none
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
603,1,1,-9.528652,1.303279,3,sub-002,LO,traintest,betas,betas,test,train,3,6,R,none,none
604,1,1,-1.943273,0.265791,3,sub-002,LO,traintest,betas,betas,test,train,3,6,R,none,none
605,1,1,-9.503316,1.299813,3,sub-002,LO,traintest,betas,betas,test,train,3,6,R,none,none
606,1,1,-10.817266,1.479528,3,sub-002,LO,traintest,betas,betas,test,train,3,6,R,none,none


In [5]:
MVPAres.correct.mean()

0.6381578947368421

In [7]:
MVPAres.testmodel.unique()

array([3, 6])

In [10]:
MVPAres.groupby(['roi', 'testmodel']).mean()

correct    output   rawdistance  distance  runno  \
roi        testmodel                                                      
LO_L       3          0.562500  0.500000 -5.203555e-02  0.303416    4.5   
           6          0.591667  0.525000 -7.715745e-03  0.126060    2.0   
LO_R       3          0.562500  0.500000  2.873767e-02  0.290023    4.5   
           6          0.500000  0.466667  2.713797e-07  0.034675    2.0   
ba-17-18_L 3          0.843750  0.468750 -5.531191e-02  0.602125    4.5   
           6          0.716667  0.450000  1.302044e-07  0.431098    2.0   
ba-17-18_R 3          0.812500  0.500000  6.353730e-02  0.679119    4.5   
           6          0.683333  0.533333  1.264045e-07  0.384545    2.0   

                      trainmodel  
roi        testmodel              
LO_L       3                 6.0  
           6                 3.0  
LO_R       3                 6.0  
           6                 3.0  
ba-17-18_L 3                 6.0  
           6                 3.0  
ba-17-18_R 3                 6.0  
           6                 3.0